# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

*Шаги для выбора локации:*

* В избранном регионе ищут месторождения, для каждого определяют значения признаков;
* Строят модель и оценивают объём запасов;
* Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
* Прибыль равна суммарной прибыли отобранных месторождений.

*Условия задачи:*

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
* Бюджет на разработку скважин в регионе — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей
* Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 1. Загрузка и подготовка данных

In [64]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from numpy.random import RandomState
from scipy import stats as st
import warnings
state = RandomState(12345)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None

In [65]:
geo_0 = pd.read_csv('geo_data_0.csv')
geo_1 = pd.read_csv('geo_data_1.csv')
geo_2 = pd.read_csv('geo_data_2.csv')

In [66]:
def df_info(df):
  print(df.info())
  display(df.head())
  display(df.describe())
  print("Уникальные значения:")
  print(df.nunique())
  print('\nПропущенные значения:')
  print(df.isna().sum())
  print('Дубликаты', df.duplicated().sum())

In [67]:
df_info(geo_0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB
None


,Unnamed: 0,id,f0,f1,f2,product
0,0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,2,409Wp,1.022732,0.151990,1.419926,85.265647
3,3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,Unnamed: 0,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,0.500419,0.250143,2.502647,92.500000
std,28867.657797,0.871832,0.504433,3.248248,44.288691
min,0.000000,-1.408605,-0.848218,-12.088328,0.000000
25%,24999.750000,-0.072580,-0.200881,0.287748,56.497507
50%,49999.500000,0.502360,0.250252,2.515969,91.849972
75%,74999.250000,1.073581,0.700646,4.715088,128.564089
max,99999.000000,2.362331,1.343769,16.003790,185.364347


Уникальные значения:
Unnamed: 0    100000
id             99990
f0            100000
f1            100000
f2            100000
product       100000
dtype: int64

Пропущенные значения:
Unnamed: 0    0
id            0
f0            0
f1            0
f2            0
product       0
dtype: int64
Дубликаты 0


In [68]:
df_info(geo_1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB
None


,Unnamed: 0,id,f0,f1,f2,product
0,0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,Unnamed: 0,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,1.141296,-4.796579,2.494541,68.825000
std,28867.657797,8.965932,5.119872,1.703572,45.944423
min,0.000000,-31.609576,-26.358598,-0.018144,0.000000
25%,24999.750000,-6.298551,-8.267985,1.000021,26.953261
50%,49999.500000,1.153055,-4.813172,2.011479,57.085625
75%,74999.250000,8.621015,-1.332816,3.999904,107.813044
max,99999.000000,29.421755,18.734063,5.019721,137.945408


Уникальные значения:
Unnamed: 0    100000
id             99996
f0            100000
f1            100000
f2            100000
product           12
dtype: int64

Пропущенные значения:
Unnamed: 0    0
id            0
f0            0
f1            0
f2            0
product       0
dtype: int64
Дубликаты 0


In [69]:
df_info(geo_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB
None


,Unnamed: 0,id,f0,f1,f2,product
0,0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,Unnamed: 0,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,0.002023,-0.002081,2.495128,95.000000
std,28867.657797,1.732045,1.730417,3.473445,44.749921
min,0.000000,-8.760004,-7.084020,-11.970335,0.000000
25%,24999.750000,-1.162288,-1.174820,0.130359,59.450441
50%,49999.500000,0.009424,-0.009482,2.484236,94.925613
75%,74999.250000,1.158535,1.163678,4.858794,130.595027
max,99999.000000,7.238262,7.844801,16.739402,190.029838


Уникальные значения:
Unnamed: 0    100000
id             99996
f0            100000
f1            100000
f2            100000
product       100000
dtype: int64

Пропущенные значения:
Unnamed: 0    0
id            0
f0            0
f1            0
f2            0
product       0
dtype: int64
Дубликаты 0


**Вывод:**

* Все датасеты хорошо подготовлены, все типы данных указаны правильно, дубликатов не имеется. В каждом по 100 000 наблюдений.
* Признак F2 очень сильно коррелирует с product
* Имеются повторяющиеся значения в столбцах ID, видимо пробы брались несколько раз в одной и той же скважине.
* У второго датасета уникальных значений в product всего 12. Наша будущая модель может слишком хорошо прогнозировать. Посмотрим...

# 2. Обучение и проверка модели


In [70]:
RMSE_list = []
predictions_list = []
score_list = []
MAE_list = []
target = []
predictions_df = pd.DataFrame()
target_df = pd.DataFrame()

for i,j in zip([geo_0,geo_1,geo_2],range(3)):
    X = i.drop(['product','id'], axis=1)
    y = i['product']
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, random_state=12345, test_size=0.25) 
    
    lr = LinearRegression(n_jobs = -1).fit(X_train, y_train)
    
    predictions = lr.predict(X_valid)
    predictions_list.append(predictions.mean())

    rmse = np.sqrt(mean_squared_error(predictions, y_valid))
    RMSE_list.append(rmse)
    
    mae = mean_absolute_error(predictions, y_valid)
    MAE_list.append(mae)

    score_list.append(lr.score(X_train, y_train))

    stock_mean = i['product'].mean()
    target.append(stock_mean)

    target_df[j] = y_valid
    predictions_df[j] = predictions

In [71]:
#Создадим таблицу с имеющимися листами, чтобы сравнить результаты
target_df = target_df.reset_index(drop=True)
scorelist = pd.DataFrame(
    data=[predictions_list, RMSE_list, score_list, MAE_list, target], 
    index=['Ср. запас сырья, тыс. баралей', 'RMSE', 'SCORE', 'MAE', 'Target'], 
    columns=['region 0','region 1','region 2'])
scorelist

,region 0,region 1,region 2
"Ср. запас сырья, тыс. баралей",92.592442,68.728500,94.965228
RMSE,37.579492,0.893066,40.030038
SCORE,0.274239,0.999625,0.196615
MAE,30.919652,0.718710,32.793091
Target,92.500000,68.825000,95.000000


**Вывод**

На данном этапе было произведено обучение линейной регрессии на данных трех регионов. В ходе обучения и проверки на валидационной выборке в каждом регионе были выявлени следующие особености:

1. Лучшие показатели модели в Регионе 1 (RMSE = 0.894, MAE = 0.721)
2. Модель в Регионе 0 и 2 показала приблизительно одинаковые результаты (RMSE = 37.9 и 40.1, соответственно что лучше значений RMSE для постоянного предсказания среднего таргета по Регионам = 44.67 и 44,66, соответственно)
3. Средние запасы сырья в месторождениях по регионам (0,1,2) = 92.5 тыс; 68.8 тыс и 95 тыс баррелей соответственно.

Несмотря на высокое качество предсказаний модели в Регионе 1, напрашивется гипотеза, что лучше сразу отбросить разработку Региона 1, поскольку средний объем нефти в месторождениях в нем меньше на ~25 тыс баррелей. Получается, что в бОльшем количестве случаев мы сможем выкачать меньше нефти и заработать меньше, чем разведав месторожожения в Регионах 0 или 2.

Однако (этот пункт я дописал уже в конце исследования) - очень качество модели говорит об обратном - добыча в регионах 0 и 2 может быть гораздо "рандомнее", а вот регион 1 мы предсказываем стабильно хорошо.

Переходим к подсчету возможной прибыли.

# 3. Подготовка к расчёту прибыли

**Ключевые метрики для рассчетов**

In [72]:
BUDGET = 10000000000      #бюджет на разработку месторождений, руб
DRILLING_COST = 50000000  #стоимость бурения одного месторождения, руб
BARREL_PROFIT = 450      #прибыль от реализации 1 барреля добытой нефти
BARREL_PROFIT_UNIT = 450000   #Доход с каждой единицы продукта 450 000 рублей

TOTAL_WELLS = 500         #количество месторождений, исследуемых при разведке региона
TOTAL_WELLS_BEST = 200 

**Дополнительные метрики для рассчетов**

Рассчитаем максимальное количемтво скважин, разработка которых укладывается в бюджет

In [73]:
real_wells = int(BUDGET / DRILLING_COST)

print('Максимально возможное количество пробуренных скважин = {:}'.format(real_wells))

Максимально возможное количество пробуренных скважин = 200


На текущий момент можно выдвинуть гипотезу о том, что разработка Регионов 0 и 2 выглядит более перспективной, так как и средние значения и распределения запасов нефти имеют лучшие значения, чем в Регионе 1.

In [74]:
drilling_zero_point = np.round(DRILLING_COST / BARREL_PROFIT_UNIT) 

print('Точка безубыточности одной скважнины = {:} тыс. добытых баррелей нефти'.format(drilling_zero_point))

Точка безубыточности одной скважнины = 111.0 тыс. добытых баррелей нефти


In [75]:
regions = [geo_0, geo_1, geo_2]

In [76]:
for i in range(len(regions)):
    print('Вероятность неокупить разрабатываемую скважину в Регионе', i, '=',
          len(regions[i].query('product < 111')) / len(regions[i]) * 100,'%')

Вероятность неокупить разрабатываемую скважину в Регионе 0 = 63.324000000000005 %
Вероятность неокупить разрабатываемую скважину в Регионе 1 = 83.463 %
Вероятность неокупить разрабатываемую скважину в Регионе 2 = 61.722 %


**Вывод**

Можно отметить, что для безубыточной разработки новой скважины необходимо переработать как минимум 112 единиц продукта. На прошлом этапе проекта выяснили, что предсказания объема нефти во всех регионах гораздо меньше этого объема.

# 4. Расчёт прибыли и рисков

На этом этапе проекта:

* напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
* посчитаем риски и прибыль для каждого региона.

Напишем функцию для расчета прибыли.

In [77]:
def revenue(target, probabilities, count):
    
    '''
    Функция принимает 3 аргумента:
    - target (истинные значения целевого признака)
    - probabilities (предсказания)
    - count (количество скважин)
    
    Функция рассчитывает и возвращает прибыль
    '''
    
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (BARREL_PROFIT_UNIT * selected.sum() - (BUDGET))

В цикле применим технику Bootstrap и посчитаем прибыль и риски для каждого из регионов:

* применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли
* найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

Результат сохраним в переменной bootstrap_results.

In [78]:
scorelist

,region 0,region 1,region 2
"Ср. запас сырья, тыс. баралей",92.592442,68.728500,94.965228
RMSE,37.579492,0.893066,40.030038
SCORE,0.274239,0.999625,0.196615
MAE,30.919652,0.718710,32.793091
Target,92.500000,68.825000,95.000000


In [79]:
#Техникой bootstrap посчитаем среднюю прибыль , 95% доверительный интервал и риск убытков для первого региона
import scipy as sp
import scipy.stats

revenues = []

for i in range(1000):
    target_subsample = target_df[0].sample(n=500, replace=True, random_state=state)
    probs_subsample = predictions_df[0][target_subsample.index]
    revenues.append(revenue(target_subsample, probs_subsample, 200))

revenues = pd.Series(revenues)
lower = revenues.quantile(0.025)
mean = revenues.mean()
risks = (revenues < 0).mean()* 100
print("Средняя прибыль:", mean/10**9)
print("2.5%-квантиль:", lower/10**9)
print("Риск: {}".format(risks))
final_revenues = []
confidence_interval = (revenues.quantile(0.025), revenues.quantile(0.975))
print('95% доверительный интервал: ', confidence_interval)
final_revenues.append(('region 1', mean/10**9, lower/10**9, confidence_interval, risks))

Средняя прибыль: 0.4257228605856449
2.5%-квантиль: -0.10209009483793653
Риск: 6.1
95% доверительный интервал:  (-102090094.83793654, 947976353.358369)


In [80]:
#Техникой bootstrap посчитаем среднюю прибыль , 95% доверительный интервал и риск убытков для второго региона
revenues = []

for i in range(1000):
    target_subsample = target_df[1].sample(n=500, replace=True, random_state=state)
    probs_subsample = predictions_df[1][target_subsample.index]
    revenues.append(revenue(target_subsample, probs_subsample, 200))

revenues = pd.Series(revenues)
lower = revenues.quantile(0.025)
mean = revenues.mean()
risks = (revenues < 0).mean()* 100
print("Средняя прибыль:", mean/10**9)
print("2.5%-квантиль:", lower/10**9)
print("Риск: {}".format(risks))
confidence_interval = (revenues.quantile(0.025), revenues.quantile(0.975))

print('95% доверительный интервал: ', confidence_interval)
final_revenues.append(('region 2', mean/10**9, lower/10**9, confidence_interval, risks))

Средняя прибыль: 0.5182637854858122
2.5%-квантиль: 0.1281232314330863
Риск: 0.3
95% доверительный интервал:  (128123231.43308629, 953612982.0669085)


In [81]:
#Техникой bootstrap посчитаем среднюю прибыль , 95% доверительный интервал и риск убытков для третьего региона
revenues = []
total_wells = 500
profit_wells = 200

for i in range(1000):
    target_subsample = target_df[2].sample(n=total_wells,replace=True, random_state=state)
    probs_subsample = predictions_df[2][target_subsample.index]
    revenues.append(revenue(target_subsample, probs_subsample, profit_wells))

revenues = pd.Series(revenues)
lower = revenues.quantile(0.025)
mean = revenues.mean()
risks = (revenues < 0).mean()* 100
#interval = interval = st.t.interval(0.95, len(revenues)-1, revenues.mean()/10**9, revenues.sem()/10**9)
print("Средняя прибыль:", mean/10**9)
print("2.5%-квантиль:", lower/10**9)
confidence_interval = (revenues.quantile(0.025), revenues.quantile(0.975))
print('95% доверительный интервал: ', confidence_interval)
print("Риск:", (revenues < 0).mean()* 100, '%')
final_revenues.append(('region 3', mean/10**9, lower/10**9, confidence_interval, risks))

Средняя прибыль: 0.42020761709581467
2.5%-квантиль: -0.11585260916000956
95% доверительный интервал:  (-115852609.16000956, 979365842.7441177)
Риск: 6.2 %


In [82]:
#Выведем все результаты в одной таблице для сравнения.
report_df = pd.DataFrame(final_revenues, columns=['Region', 'Mean Revenue', 'Quantile 2.5%', 'Interval 95%', 'Risks %'])
cm = sns.light_palette("green", as_cmap=True)
s = report_df.style.background_gradient(cmap=cm)
s

,Region,Mean Revenue,Quantile 2.5%,Interval 95%,Risks %
0,region 1,0.425723,-0.102090,"(-102090094.83793654, 947976353.358369)",6.100000
1,region 2,0.518264,0.128123,"(128123231.43308629, 953612982.0669085)",0.300000
2,region 3,0.420208,-0.115853,"(-115852609.16000956, 979365842.7441177)",6.200000



**Вывод:**

* Второй регион показывает хорошие результаты. Но так как мы знаем, что второй регион слишком рискованный, слишком странные у него данные - мы не можем его выбрать.
* Поэтому для бурения будет лучше выбрать первый регион. Но учитывая оценки прогнозирования, я бы вообще не принимал никаких серьезных решений на основе полученных результатов.


# Общий вывод:

* Мы получили три датасета - три разные региона для прогнозирования объема сырья и прибыли. Все датасеты были хорошо подготовлены, не имелось никаких дубликатов, пустых значений. Но второй датасет оказался сильно отличимым от других, у него в столбце product были лишь 12 уникальных значений. Что позволило нашей модели хорошо делать прогнозы по второму региону ,поэтому нам нельзя использовать эту модель и сам регион. Слишком рискованно.

* Исходя из всех результатов моделей мы получили такие результаты:

  * Регион 1: RMSE : 37.5 ; Score 0.27
  * Регион 2: RMSE : 0.89 ; Score 0.99
  * Регион 3: RMSE : 40.02 ; Score 0.19

* Провели расчет прибыли и пришли к выводу, что минимальный объем сырья должен составлять 111 тыс барелей. Наши средние показатели трех регионов не доходят до минимального порога.

Исходя из всех полученных данных: второй регион имеет наибольшие шансы на получение прибыли. Но учитывая тот факт, что все модели показали ужасные результаты, и из худшего мы выбрали наибольшее лучший, я бы не советовал использовать эти данные для принятия окончательных решений.
